In [2]:
import sqlite3
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

pd.set_option('display.max_columns', None)

In [3]:
conn = sqlite3.Connection('./congress-data_v2.4.db')

In [4]:
cur = conn.cursor()
cur.execute('CREATE INDEX idx_full_texts_file_chamber ON full_texts(file_chamber)')
conn.commit()

In [5]:
cur = conn.cursor()

cur.execute('drop table if exists cleaned_bills')
conn.commit()

create_cleaned = """
    create table if not exists cleaned_bills (
        id integer primary key,
        title text,
        text text,
        summary text,
        ft_id integer,
        foreign key (ft_id) references ft(id)
    )
"""

cur.execute(create_cleaned)
conn.commit()

In [6]:
def remove_stopwords_and_stem(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(stemmed_words)

In [7]:
"""
Takes about an hour to run.
"""

insert = """
    insert into cleaned_bills (
        title,
        text,
        summary,
        ft_id
    ) values (?, ?, ?, ?)
"""

df = pd.read_sql_query(f"""
    select 
        ft.id as ft_id, 
        coalesce(ft.title, '') as title,
        coalesce(ft.text, '') as text,
        coalesce(bs.summary_text, '') as summary_text
    from full_texts ft
        left join bill_summaries bs
        on ft.summaries_match = bs.id
""", conn, chunksize=1000)

for i, chunk in enumerate(df):
    print('Processing:', i)

    chunk.title = chunk.title.apply(remove_stopwords_and_stem)
    chunk.text = chunk.text.apply(remove_stopwords_and_stem)
    chunk.summary_text = chunk.summary_text.apply(remove_stopwords_and_stem)

    cur.executemany(insert, chunk[['title', 'text', 'summary_text', 'ft_id']].values)
    conn.commit()

Processing: 0
Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
Processing: 6
Processing: 7
Processing: 8
Processing: 9
Processing: 10
Processing: 11
Processing: 12
Processing: 13
Processing: 14
Processing: 15
Processing: 16
Processing: 17
Processing: 18
Processing: 19
Processing: 20
Processing: 21
Processing: 22
Processing: 23
Processing: 24
Processing: 25
Processing: 26
Processing: 27
Processing: 28
Processing: 29
Processing: 30
Processing: 31
Processing: 32
Processing: 33
Processing: 34
Processing: 35
Processing: 36
Processing: 37
Processing: 38
Processing: 39
Processing: 40
Processing: 41
Processing: 42
Processing: 43
Processing: 44
Processing: 45
Processing: 46
Processing: 47
Processing: 48
Processing: 49
Processing: 50
Processing: 51
Processing: 52
Processing: 53
Processing: 54
Processing: 55
Processing: 56
Processing: 57
Processing: 58
Processing: 59
Processing: 60
Processing: 61
Processing: 62
Processing: 63
Processing: 64
Processing: 65
Processing: 66
Proce

In [11]:
drop_table = "drop table congress_bm25"
cur = conn.cursor()

cur.execute(drop_table)
conn.commit()


create_virtual_table = """
    create virtual table congress_bm25 
    using fts5(
        summary_text, 
        title, 
        text,
        ft_id UNINDEXED,

        tokenize='porter'
    )
"""

cur = conn.cursor()

cur.execute(create_virtual_table)
conn.commit()

insert_into_virtual_table = """
    insert into congress_bm25(
        summary_text, 
        title, 
        text, 
        ft_id
    )
    select summary, title, text, ft_id
    from cleaned_bills
"""

cur.execute(insert_into_virtual_table)
conn.commit()

In [5]:
cur = conn.cursor()
cur.execute('CREATE INDEX idx_bert_embeddings_full_text_id ON bert_embeddings(full_text_id)')
conn.commit()

In [1]:
cur = conn.cursor()
cur.execute('VACUUM')
conn.commit()